In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [2]:
{'lr': 0.00034439316653688684,
 'layers': 3,
 'step_size': 11,
 'gamma': 0.761795969995615,
 'bptt': 19,
 'dropout': 0.1227497445640586}

{'lr': 0.00034439316653688684,
 'layers': 3,
 'step_size': 11,
 'gamma': 0.761795969995615,
 'bptt': 19,
 'dropout': 0.1227497445640586}

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [4]:
dataset = pd.read_csv('/home/urwa/Documents/side_projects/urban/data/featureData/jfk.csv')

In [5]:
dataset.shape

(8757, 1049)

In [6]:
dataset.head(3)

,Date,Hour,1,10,100,101,102,106,107,108,...,91_lag_3,92_lag_3,93_lag_3,94_lag_3,95_lag_3,96_lag_3,97_lag_3,98_lag_3,99_lag_3,arrival_lag_3
0,2018-01-01,3,0,0,0,0,0,0,0,0,...,1.0,1.0,0.0,1.0,6.0,0.0,1.0,0.0,0.0,6.0
1,2018-01-01,4,0,3,0,0,1,0,0,1,...,4.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,6.0
2,2018-01-01,5,0,4,0,0,1,2,3,1,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0


In [7]:
lag_columns = [c for c in dataset.columns if 'lag' in c]
len(lag_columns)

777

In [8]:
dataset = dataset[[c for c in dataset.columns if c not in lag_columns]]
dataset.shape

(8757, 272)

In [9]:
DateColumns = ['Date']

ext_columns = ['Dow', 'arrival','maxtemp', 'mintemp', 'avgtemp', 'departure', 'hdd',
       'cdd', 'participation', 'newsnow', 'snowdepth', 'ifSnow']

targetColumns = [c for c in dataset.columns if c not in ext_columns and \
                c not in DateColumns and c not in lag_columns and c != 'Hour']
len(targetColumns)

258

In [10]:
features_cols = [c for c in dataset.columns if c not in targetColumns and c not in DateColumns]
len(features_cols)

13

In [11]:
sep = int(0.75*len(dataset))
print(sep)


trainData = dataset[:sep]
testData = dataset[sep:]

print(trainData.shape)
print(testData.shape)

6567
(6567, 272)
(2190, 272)


In [12]:
X_train = trainData[features_cols].values
X_train = torch.tensor(X_train).float().to(device)
print(X_train.shape)

y_train = trainData[targetColumns].values
y_train = torch.tensor(y_train).float().to(device)
print(y_train.shape)

X_test = testData[features_cols].values
X_test = torch.tensor(X_test).float().to(device)
print(X_test.shape)

y_test = testData[targetColumns].values
y_test = torch.tensor(y_test).float().to(device)
print(y_test.shape)

torch.Size([6567, 13])
torch.Size([6567, 258])
torch.Size([2190, 13])
torch.Size([2190, 258])


In [13]:
def create_inout_sequences(x,y, tw):
    inout_seq = []
    L = len(x)
    for i in range(L-tw):
        train_seq_x = x[i:i+tw]
        train_seq_y = y[i:i+tw]
#         train_seq = torch.cat((train_seq_x,train_seq_y),axis=1)
        
#         train_label = y[i+tw:i+tw+1]
        train_label = y[i+1:i+tw+1]
        inout_seq.append((train_seq_x, train_seq_y ,train_label))
    return inout_seq

In [14]:
bptt = 19

In [15]:
train_inout_seq = create_inout_sequences(X_train,y_train, bptt)

In [16]:
train_inout_seq[0][0].shape,train_inout_seq[0][1].shape, train_inout_seq[0][2].shape

(torch.Size([19, 13]), torch.Size([19, 258]), torch.Size([19, 258]))

In [17]:
test_inout_seq = create_inout_sequences(X_test,y_test, bptt)

In [18]:
class LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_size=100, num_layers=1, dropout=0):
        super().__init__()
                
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.hidden_cell = (torch.zeros(num_layers,1,self.hidden_size).to(device),
                    torch.zeros(num_layers,1,self.hidden_size).to(device))
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, dropout=dropout)


    def forward(self, x):
        self.hidden_cell = (torch.zeros(self.num_layers,1,self.hidden_size).to(device),
                    torch.zeros(self.num_layers,1,self.hidden_size).to(device))
           
        lstm_out, self.hidden_cell = self.lstm(x.view(len(x) ,1, -1), self.hidden_cell)
        
        return lstm_out, self.hidden_cell

In [19]:
class GraphPrediction(nn.Module):
    def __init__(self, feat_size=1, hidden_layer_size=100, network_size=1, layers=1, communities=10, ensembles=1, dropout=0):
        super().__init__()
        
        # aggregation
        self.attachment_matrix = torch.nn.Parameter(torch.randn(network_size,communities))
        self.attachment_matrix.requires_grad = True
        
        lstm_input = communities + feat_size
        
        self.ensembles = ensembles
        self.lstms = nn.ModuleList()
        self.linears = nn.ModuleList()
        for i in range(ensembles):
             self.lstms.append(LSTM(input_size=lstm_input, hidden_size=hidden_layer_size, num_layers=layers))
             self.linears.append(nn.Linear(hidden_layer_size, network_size))
            

    def forward(self, input_seq, feat):
        
        w = F.softmax(self.attachment_matrix, dim=1)
        x = torch.matmul(input_seq, self.attachment_matrix)
        x = torch.cat((x,feat),axis=1)
        x = x.view(len(input_seq) ,1, -1)
        
        predictions = []
        for i in range(self.ensembles):
            if torch.randn(1) < 0.7 or i==0 or not self.training:
                lstm_out, self.hidden_cell = self.lstms[i](x)
                y = self.linears[i](lstm_out.view(len(input_seq), -1))
                predictions.append(y)
        
        predictions = torch.stack(predictions)
#         print(predictions.shape)
        return predictions

In [20]:
def evaluate(model):
    model.eval()
    prediction = []
    with torch.no_grad():
        for feat,seq, labels in test_inout_seq:
#             model.hidden = (torch.zeros(layers, 1, model.hidden_layer_size),
#                             torch.zeros(layers, 1, model.hidden_layer_size))
            y = model(seq,feat).mean(dim=0)[-1]
    
            prediction.append(y)

    y_test_ = torch.stack([labels[-1] for feat,seq, labels in test_inout_seq], axis=0).detach().cpu().numpy()
    y_pred_ = torch.stack(prediction).detach().cpu().numpy()

    r2 = r2_score(y_test_, y_pred_, multioutput='variance_weighted')
    rmse = mean_squared_error(y_test_, y_pred_)
    mae = mean_absolute_error(y_test_, y_pred_)
#     print("r2: ",r2)
    return (r2, rmse, mae)

In [21]:
layers = 3
communities = 24
network_size = len(targetColumns)
feat_size = len(features_cols)
ensembles=10
dropout= 0.1227497445640586

model = GraphPrediction(feat_size = feat_size, hidden_layer_size=100,
             network_size=network_size, layers=layers,
            communities=communities, ensembles=ensembles, dropout=dropout).to(device)



In [22]:
loss_function = nn.L1Loss()   
optimizer = torch.optim.Adam(model.parameters(), lr=0.00034439316653688684)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=11, gamma=0.762)

In [23]:
epochs = 150


for i in range(epochs):
    model.train()
    for feat,seq, labels in train_inout_seq:
        optimizer.zero_grad()
#         model.hidden_cell = (torch.zeros(layers, 1, model.hidden_layer_size).to(device),
#                         torch.zeros(layers, 1, model.hidden_layer_size).to(device))

        y_pred = model(seq, feat)
        labels = labels.repeat(y_pred.shape[0],1,1)
        
        single_loss = loss_function(y_pred, labels)
        single_loss.backward()
        optimizer.step()
        
    scheduler.step()
#     if i%10 == 1:
    r2, rmse, mae = evaluate(model)
    print(f'epoch: {i:3} loss: {single_loss.item():10.8f} r2: {r2:5.3f} rmse: {rmse:5.3f} mae: {mae:5.3f}')
    torch.save(model.state_dict(), 'jfk.pt')

print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')

epoch:   0 loss: 1.09996319 r2: 0.424 rmse: 4.914 mae: 1.211
epoch:   1 loss: 1.08950555 r2: 0.502 rmse: 4.255 mae: 1.128
epoch:   2 loss: 1.06901085 r2: 0.502 rmse: 4.252 mae: 1.129
epoch:   3 loss: 1.06093681 r2: 0.518 rmse: 4.113 mae: 1.111
epoch:   4 loss: 1.03974140 r2: 0.543 rmse: 3.898 mae: 1.086
epoch:   5 loss: 1.03118396 r2: 0.556 rmse: 3.791 mae: 1.070
epoch:   6 loss: 1.03019822 r2: 0.560 rmse: 3.753 mae: 1.068
epoch:   7 loss: 1.02515256 r2: 0.561 rmse: 3.750 mae: 1.069
epoch:   8 loss: 1.01914835 r2: 0.566 rmse: 3.702 mae: 1.063
epoch:   9 loss: 1.02117848 r2: 0.567 rmse: 3.698 mae: 1.064
epoch:  10 loss: 1.01926136 r2: 0.567 rmse: 3.694 mae: 1.063
epoch:  11 loss: 1.01369584 r2: 0.571 rmse: 3.666 mae: 1.059
epoch:  12 loss: 1.01095557 r2: 0.569 rmse: 3.675 mae: 1.062
epoch:  13 loss: 1.01351810 r2: 0.571 rmse: 3.662 mae: 1.060
epoch:  14 loss: 1.00872242 r2: 0.571 rmse: 3.665 mae: 1.061
epoch:  15 loss: 1.00321805 r2: 0.573 rmse: 3.649 mae: 1.061
epoch:  16 loss: 1.01278

epoch: 135 loss: 0.95557272 r2: 0.571 rmse: 3.661 mae: 1.059
epoch: 136 loss: 0.94719929 r2: 0.571 rmse: 3.661 mae: 1.059
epoch: 137 loss: 0.94951469 r2: 0.571 rmse: 3.661 mae: 1.059
epoch: 138 loss: 0.95181382 r2: 0.571 rmse: 3.661 mae: 1.058
epoch: 139 loss: 0.95426887 r2: 0.571 rmse: 3.661 mae: 1.059
epoch: 140 loss: 0.95426911 r2: 0.571 rmse: 3.662 mae: 1.058
epoch: 141 loss: 0.95195401 r2: 0.571 rmse: 3.662 mae: 1.058
epoch: 142 loss: 0.95168722 r2: 0.571 rmse: 3.662 mae: 1.058
epoch: 143 loss: 0.95169663 r2: 0.571 rmse: 3.666 mae: 1.059
epoch: 144 loss: 0.95087862 r2: 0.570 rmse: 3.667 mae: 1.059
epoch: 145 loss: 0.95364934 r2: 0.570 rmse: 3.668 mae: 1.059
epoch: 146 loss: 0.95414239 r2: 0.570 rmse: 3.668 mae: 1.059
epoch: 147 loss: 0.95155489 r2: 0.570 rmse: 3.668 mae: 1.059
epoch: 148 loss: 0.95277727 r2: 0.570 rmse: 3.669 mae: 1.059
epoch: 149 loss: 0.94753027 r2: 0.570 rmse: 3.669 mae: 1.059
epoch: 149 loss: 0.9475302696


In [24]:
# bezt value 0.577

In [25]:
evaluate(model)

(0.5702683714827709, 3.6686943, 1.0587766)

In [26]:
attachment = torch.argmax(F.softmax(model.attachment_matrix, dim=1), dim=1).detach().cpu().numpy().astype(str)
community_assignment = dict(zip(targetColumns, attachment))
community_assignment

{'1': '18',
 '10': '6',
 '100': '1',
 '101': '3',
 '102': '4',
 '106': '19',
 '107': '4',
 '108': '15',
 '109': '1',
 '11': '2',
 '110': '20',
 '111': '7',
 '112': '13',
 '113': '19',
 '114': '18',
 '115': '17',
 '116': '5',
 '117': '10',
 '118': '1',
 '119': '9',
 '12': '23',
 '120': '6',
 '121': '17',
 '122': '17',
 '123': '18',
 '124': '13',
 '125': '3',
 '126': '18',
 '127': '13',
 '128': '11',
 '129': '7',
 '13': '22',
 '130': '9',
 '131': '19',
 '133': '7',
 '134': '9',
 '135': '13',
 '136': '4',
 '137': '19',
 '138': '16',
 '139': '19',
 '14': '16',
 '140': '12',
 '141': '5',
 '142': '7',
 '143': '2',
 '144': '7',
 '145': '15',
 '146': '6',
 '147': '17',
 '148': '14',
 '149': '13',
 '15': '15',
 '150': '11',
 '151': '12',
 '152': '2',
 '153': '7',
 '154': '17',
 '155': '17',
 '156': '19',
 '157': '1',
 '158': '4',
 '159': '16',
 '16': '10',
 '160': '6',
 '161': '6',
 '162': '0',
 '163': '22',
 '164': '9',
 '165': '22',
 '166': '18',
 '167': '4',
 '168': '18',
 '169': '13',
 '17'

In [27]:
import json

with open('jfk_attachment.json', 'w') as fp:
    json.dump(community_assignment, fp)

In [28]:
len(set(attachment))

24

In [29]:
# 2 linear
# 0.47

In [30]:
# 1 linear
# 0.51

In [31]:
# 1 linear + RELU
# 0.50

In [32]:
# 1 linear bptt = 24
# 0.52

In [33]:
#ensemble
#0.53228

In [34]:
#  TODO
# other hubs
# ensemble model